### Importing and installing necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Assignment 3.1

### Wikipage tables are copied to dataframe

In [2]:
table=pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695')
len(table)

3

In [17]:
table[0].head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [4]:
postcodeToronto=table[0]

In [5]:
postcodeToronto.shape

(288, 3)

### Dropping rows where Borough = 'Not assigned'

In [18]:
#postcodeToronto.drop(postcodeToronto[postcodeToronto.Borough == 'Not assigned'].index, inplace=True)
postcodeToronto=postcodeToronto[postcodeToronto.Borough != 'Not assigned']

In [19]:
postcodeToronto.shape

(211, 3)

In [21]:
postcodeToronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### grouping same postcodes into a single row 

In [22]:
Toronto_df=postcodeToronto.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Processing rows where Neighbourhood=Not assigned

In [9]:
for index, row in Toronto_df.iterrows():
    if row['Neighbourhood']=='Not assigned':
        Toronto_df.iloc[index, Toronto_df.columns.get_loc('Neighbourhood')] = row['Borough']

#Toronto_df.at[index, 'Neighbourhood'] = row['Borough'] # This should also work

In [23]:
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
Toronto_df.shape

(103, 3)

# Assignment 3.2

### Reading latitude-longitude data from the csv file

In [25]:
latlon_df=pd.read_csv('Geospatial_Coordinates.csv')
latlon_df.columns=['Postcode1', 'Latitude','Longitude']

In [26]:
latlon_df.head()

,Postcode1,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### joining lat-long columns into main postcode datraframe

In [27]:
postcodeLatLon_df = pd.concat([Toronto_df, latlon_df],  axis=1, join='inner')
postcodeLatLon_df.head()

,Postcode,Borough,Neighbourhood,Postcode1,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Dropping duplicate column

In [28]:
postcodeLatLon_df.drop('Postcode1', axis=1, inplace=True)

In [29]:
postcodeLatLon_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Assignment 3.3

### 1. Toronto neighbourhood map

In [32]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [33]:
torontoNeighborhoods=postcodeLatLon_df
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontoNeighborhoods['Latitude'],
                                           torontoNeighborhoods['Longitude'],
                                           torontoNeighborhoods['Borough'],
                                           torontoNeighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### 2. Foursquare credentials and get vanues

In [34]:
CLIENT_ID = 'xxx' # Foursquare ID removed
CLIENT_SECRET = 'yyy' # Foursquare Secret removed
VERSION = '20200328' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TDGZX32NWWDKR3K5IJGHX0RVDUIRZRCS2YKFCFNFHQ2IJXWE
CLIENT_SECRET:BDZ3Y4FYWZWBKNLBXNGARGUKX5OMMDXMUR5CLRK5OLODKD1H


####  Get vanues with the function getNearbyVanues 

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
toronto_venues = getNearbyVenues(names=torontoNeighborhoods['Neighbourhood'],
                                   latitudes=torontoNeighborhoods['Latitude'],
                                   longitudes=torontoNeighborhoods['Longitude']
                                  )
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [48]:
print("Our venues dataset includes", toronto_venues.shape[0], 'venues, each has', toronto_venues.shape[1], 'attributes')
print('There are', len(toronto_venues['Venue Category'].unique()), 'uniques categories in total')

Our venues dataset includes 2188 venues, each has 7 attributes
There are 264 uniques categories in total


In [49]:
toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Adelaide, King, Richmond                             100
Commerce Court, Victoria Hotel                       100
St. James Town                                       100
Harbourfront East, Toronto Islands, Union Station    100
Design Exchange, Toronto Dominion Centre             100
Name: Venue, dtype: int64

###  3. Analyze neighbourhood

In [69]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(0, 'NeighborhoodName', toronto_venues['Neighborhood'])
toronto_grouped = toronto_onehot.groupby('NeighborhoodName').sum().reset_index()
toronto_grouped.head()

,NeighborhoodName,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,2,0,0,1,0,2,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,2,0,0,0,1,1,1,0,0,0,0,5,0,0,0,0,0,0,0,0,0,2,0,7,0,0,0,0,0,1,0,0,2,0,0,2,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['NeighborhoodName']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Bakery,Thai Restaurant,Bar,Cosmetics Shop,Lounge,Concert Hall,Gym
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Sandwich Place,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Pizza Place,Comfort Food Restaurant,Dessert Shop,Eastern European Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Skating Rink,Coffee Shop,Airport Terminal,Deli / Bodega


### 4. Neighbourhood clustering with KMenas

In [75]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('NeighborhoodName', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


neighborhoods_venues_sorted['Cluster label'] = kmeans.labels_
torontoNeighborhoods.columns=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
toronto_merged = pd.merge(neighborhoods_venues_sorted, torontoNeighborhoods, how='left', on='Neighborhood', validate="1:1")
toronto_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster label,Postcode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Bakery,Thai Restaurant,Bar,Cosmetics Shop,Lounge,Concert Hall,Gym,1,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,3,M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop,3,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Sandwich Place,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Pizza Place,Comfort Food Restaurant,Dessert Shop,Eastern European Restaurant,3,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood, Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Skating Rink,Coffee Shop,Airport Terminal,Deli / Bodega,3,M8W,Etobicoke,43.602414,-79.543484


### 5. Create map

In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neigh, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Postcode'], toronto_merged['Cluster label']):
    label = folium.Popup(str(neigh) + '(' + str(pc) + '): Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


### 6. Analyze clusters

In [80]:
toronto_merged.loc[toronto_merged['Cluster label'] == 0, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Thai Restaurant,Indian Restaurant,Pub,Butcher,Sushi Restaurant,Liquor Store
8,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Beer Bar,Bakery,Farmers Market,Cheese Shop,Café,Beach
11,Café,Breakfast Spot,Nightclub,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
15,Coffee Shop,Restaurant,Café,Bakery,Italian Restaurant,Pizza Place,Chinese Restaurant,Pub,Liquor Store,Beer Store
20,Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Dessert Shop,Pizza Place,Bakery


In [81]:
toronto_merged.loc[toronto_merged['Cluster label'] == 1, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Coffee Shop,Restaurant,Café,Bakery,Thai Restaurant,Bar,Cosmetics Shop,Lounge,Concert Hall,Gym
27,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
32,Coffee Shop,Café,Hotel,Restaurant,Bar,Italian Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,American Restaurant
44,Coffee Shop,Café,Restaurant,Hotel,Steakhouse,American Restaurant,Seafood Restaurant,Gym,Gastropub,Japanese Restaurant
81,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Diner,Breakfast Spot,Bakery,Cosmetics Shop,Clothing Store


In [82]:
toronto_merged.loc[toronto_merged['Cluster label'] == 2, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Bakery,Women's Store,Bank,Juice Bar,Convenience Store,Electronics Store
78,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ramen Restaurant,Thai Restaurant,Restaurant


In [83]:
toronto_merged.loc[toronto_merged['Cluster label'] == 3, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,Park,Playground,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
3,Grocery Store,Sandwich Place,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Pizza Place,Comfort Food Restaurant,Dessert Shop,Eastern European Restaurant
4,Pizza Place,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Skating Rink,Coffee Shop,Airport Terminal,Deli / Bodega
5,Coffee Shop,Bank,Middle Eastern Restaurant,Fried Chicken Joint,Sandwich Place,Bridal Shop,Diner,Restaurant,Deli / Bodega,Supermarket


In [84]:
toronto_merged.loc[toronto_merged['Cluster label'] == 4, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Coffee Shop,Italian Restaurant,Burger Joint,Japanese Restaurant,Thai Restaurant,Sandwich Place,Gym / Fitness Center,Salad Place,Department Store,Café
50,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Sporting Goods Shop,Restaurant,Brewery
